In [1]:
# Cell 1: Install dependencies
# Run this cell first to install required libraries
!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00


In [8]:
# Cell 2: Import libraries and load YOLO model
from ultralytics import YOLO
import cv2
from google.colab import files
import os
import pandas as pd

# Load pre-trained YOLOv8 nano model (downloads automatically on first run)
model = YOLO('yolov8n.pt')  # Uses 'person' class (id 0) for detection

In [3]:
# Cell 3: Upload your video file
# Upload a video file (e.g., .mp4) from your local machine
uploaded = files.upload()

# Get the path to the uploaded video (assumes single file upload)
video_path = list(uploaded.keys())[0]
print(f"Uploaded video: {video_path}")

Saving Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4 to Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4
Uploaded video: Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4


In [17]:
# Cell 4: Process the video with line crossing + total unique people
# Combines middle line crossing (unique entries/exits) with total unique tracks seen.
# Virtual horizontal line at mid-frame as 'gate'.
# Collects all unique track IDs for total unique people.
# Counts unique crossings: UP->DOWN (entries), DOWN->UP (exits).
# Overlays line, running crossing counts, and total unique on video.

# Virtual line setup (horizontal at middle; adjust line_y for gate)
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
line_y = height // 2

# Output video setup
output_path = 'output_combined_metrics.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
track_centroids_prev = {}  # {track_id: prev_y}
all_unique_tracks = set()  # All unique track IDs seen
crossed_up_to_down = set()  # Unique UP->DOWN crossers
crossed_down_to_up = set()  # Unique DOWN->UP crossers

print("Processing video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Run YOLO tracking
    results = model.track(frame, persist=True, tracker="bytetrack.yaml", verbose=False)

    # Current frame data
    track_centroids_curr = {}

    for result in results:
        if result.boxes is not None and result.boxes.id is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            track_ids = result.boxes.id.cpu().numpy().astype(int)
            cls_ids = result.boxes.cls.cpu().numpy().astype(int)
            for i, (box, track_id, cls_id) in enumerate(zip(boxes, track_ids, cls_ids)):
                if model.names[cls_id] == 'person':
                    all_unique_tracks.add(track_id)  # Collect all unique
                    x1, y1, x2, y2 = map(int, box)
                    centroid_y = (y1 + y2) // 2
                    track_centroids_curr[track_id] = centroid_y

    # Detect crossings
    for track_id, curr_y in track_centroids_curr.items():
        if track_id in track_centroids_prev:
            prev_y = track_centroids_prev[track_id]
            if prev_y < line_y and curr_y >= line_y and track_id not in crossed_up_to_down:
                crossed_up_to_down.add(track_id)
            elif prev_y > line_y and curr_y <= line_y and track_id not in crossed_down_to_up:
                crossed_down_to_up.add(track_id)
        track_centroids_prev[track_id] = curr_y

    # Prune inactive tracks
    inactive_tracks = [tid for tid in track_centroids_prev if tid not in track_centroids_curr]
    if frame_count % 50 == 0:
        for tid in inactive_tracks:
            del track_centroids_prev[tid]

    # Running metrics
    running_unique_total = len(all_unique_tracks)
    running_up_to_down = len(crossed_up_to_down)
    running_down_to_up = len(crossed_down_to_up)

    # Overlay: line and running metrics
    cv2.line(frame, (0, line_y), (width, line_y), (255, 0, 0), 1)
    cv2.putText(frame, f'Unique Total: {running_unique_total}', (5, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
    cv2.putText(frame, f'UP->DOWN: {running_up_to_down}', (5, 55),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.putText(frame, f'DOWN->UP: {running_down_to_up}', (5, 85),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Draw boxes colored by crossing
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            track_ids = result.boxes.id.cpu().numpy().astype(int) if result.boxes.id is not None else None
            cls_ids = result.boxes.cls.cpu().numpy().astype(int)
            for i, (box, cls_id) in enumerate(zip(boxes, cls_ids)):
                if model.names[cls_id] == 'person':
                    x1, y1, x2, y2 = map(int, box)
                    track_id = int(track_ids[i]) if track_ids is not None else -1
                    color = (0, 255, 0) if track_id in crossed_up_to_down else (0, 0, 255) if track_id in crossed_down_to_up else (255, 0, 0)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Final metrics
total_unique_people = len(all_unique_tracks)
total_up_to_down = len(crossed_up_to_down)
total_down_to_up = len(crossed_down_to_up)

print(f"\nProcessing complete!")
print(f"Total frames: {frame_count}")
print(f"Total unique people in video: {total_unique_people}")
print(f"Unique people crossing UP->DOWN: {total_up_to_down}")
print(f"Unique people crossing DOWN->UP: {total_down_to_up}")
print(f"Output video saved as: {output_path}")

Processing video...

Processing complete!
Total frames: 341
Total unique people in video: 109
Unique people crossing UP->DOWN: 15
Unique people crossing DOWN->UP: 15
Output video saved as: output_combined_metrics.mp4


In [9]:
# Cell 5: Save metrics to CSV
data = {
    'Metric': ['Total Unique People', 'Total UP to DOWN Crossings', 'Total DOWN to UP Crossings'],
    'Value': [total_unique_people, total_up_to_down, total_down_to_up]
}
df = pd.DataFrame(data)
csv_path = 'people_metrics_summary.csv'
df.to_csv(csv_path, index=False)
print(f"\nMetrics saved to: {csv_path}")
print(df)


Metrics saved to: people_metrics_summary.csv
                       Metric  Value
0         Total Unique People    110
1  Total UP to DOWN Crossings     15
2  Total DOWN to UP Crossings     15


In [18]:
# Cell 6: Download files
# files.download(csv_path)
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>